In [28]:
import os

root_dir = "wiki-small/"

output_trec_file = "ejemplo.trec"

def enumerate_html_files(directory):
    document_id = 1
    with open(output_trec_file, 'w') as trec_file:
        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith(".html"):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r', encoding='utf-8') as html_file:
                        content = html_file.read()
                        trec_file.write(f"<DOC>\n")
                        trec_file.write(f"<DOCNO>{document_id}</DOCNO>\n")
                        trec_file.write(f"<TEXT>\n")
                        trec_file.write(f"{content}\n")
                        trec_file.write(f"</TEXT>\n")
                        trec_file.write(f"</DOC>\n")
                        # Incrementar el ID del documento
                        document_id += 1

enumerate_html_files(root_dir)



In [29]:
import pyterrier as pt

if not pt.started():
    pt.init()

path_to_collection = "ejemplo.trec"
root_dir = "wiki-small/"

indexer = pt.FilesIndexer("./index", verbose=True, overwrite=True, meta={"docno":20, "filename":512})
indexref =indexer.index(root_dir)
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())


Number of documents: 6043
Number of terms: 164940
Number of postings: 1689407
Number of fields: 0
Number of tokens: 3128731
Field names: []
Positions:   false



In [39]:
# ad-hoc retrieval
import numpy as np
br =  pt.BatchRetrieve(index, num_results=50, wmodel="TF_IDF", metadata=["filename"])
results = br.search("software")
results

,qid,docid,filename,rank,score,query
0,1,5855,wiki-small//en/articles/j/f/i/JFin_1830.html,0,6.463159,software
1,1,3570,wiki-small//en/articles/s/t/e/Stellarium.html,1,6.448018,software
2,1,1449,wiki-small//en/articles/a/u/t/Autodesk_Develop...,2,6.426105,software
3,1,5175,wiki-small//en/articles/r/o/x/ROX_Desktop_b4f8...,3,6.350863,software
4,1,5941,wiki-small//en/articles/k/a/o/KAOS_(software_d...,4,6.291252,software
5,1,2498,wiki-small//en/articles/c/d/r/CDRWIN_b7e8.html,5,6.170152,software
6,1,3881,wiki-small//en/articles/m/i/c/Microtuner.html,6,6.152080,software
7,1,1030,wiki-small//en/articles/p/p/3/PP3_444f.html,7,6.149680,software
8,1,3766,wiki-small//en/articles/f/o/r/Fortress_(softwa...,8,6.119815,software
9,1,1272,wiki-small//en/articles/y/a/s/Yass_(software)....,9,6.050842,software


In [31]:
tfidf = np.array(results["filename"])
tfidf.tolist()

['wiki-small//en/articles/j/f/i/JFin_1830.html',
 'wiki-small//en/articles/s/t/e/Stellarium.html',
 'wiki-small//en/articles/a/u/t/Autodesk_Developer_Network_cf4f.html',
 'wiki-small//en/articles/r/o/x/ROX_Desktop_b4f8.html',
 'wiki-small//en/articles/k/a/o/KAOS_(software_development)_665f.html',
 'wiki-small//en/articles/c/d/r/CDRWIN_b7e8.html',
 'wiki-small//en/articles/m/i/c/Microtuner.html',
 'wiki-small//en/articles/p/p/3/PP3_444f.html',
 'wiki-small//en/articles/f/o/r/Fortress_(software).html',
 'wiki-small//en/articles/y/a/s/Yass_(software).html',
 'wiki-small//en/articles/s/c/r/Scramdisk.html',
 'wiki-small//en/articles/e/l/e/Electronic_data_capture.html',
 'wiki-small//en/articles/p/i/r/Piriform.html',
 'wiki-small//en/articles/m/i/k/MiKTeX_fd58.html',
 'wiki-small//en/articles/n/+/_/N+.html',
 'wiki-small//en/articles/s/o/f/SoftRAM_b9df.html',
 'wiki-small//en/articles/c/l/x/CLX_(Common_Lisp)_1911.html',
 'wiki-small//en/articles/u/n/i/Universal_Robotics_0bd1.html',
 'wiki-sm

In [43]:
# ad-hoc retrieval
br =  pt.BatchRetrieve(index, num_results=50, wmodel="BM25", metadata=["filename"])
results = br.search("software")
results

,qid,docid,filename,rank,score,query
0,1,5855,wiki-small//en/articles/j/f/i/JFin_1830.html,0,11.724293,software
1,1,3570,wiki-small//en/articles/s/t/e/Stellarium.html,1,11.696828,software
2,1,1449,wiki-small//en/articles/a/u/t/Autodesk_Develop...,2,11.657075,software
3,1,5175,wiki-small//en/articles/r/o/x/ROX_Desktop_b4f8...,3,11.520586,software
4,1,5941,wiki-small//en/articles/k/a/o/KAOS_(software_d...,4,11.412450,software
5,1,2498,wiki-small//en/articles/c/d/r/CDRWIN_b7e8.html,5,11.192772,software
6,1,3881,wiki-small//en/articles/m/i/c/Microtuner.html,6,11.159989,software
7,1,1030,wiki-small//en/articles/p/p/3/PP3_444f.html,7,11.155637,software
8,1,3766,wiki-small//en/articles/f/o/r/Fortress_(softwa...,8,11.101461,software
9,1,1272,wiki-small//en/articles/y/a/s/Yass_(software)....,9,10.976343,software


In [44]:
bm25 = np.array(results["filename"])
bm25.tolist()

['wiki-small//en/articles/j/f/i/JFin_1830.html',
 'wiki-small//en/articles/s/t/e/Stellarium.html',
 'wiki-small//en/articles/a/u/t/Autodesk_Developer_Network_cf4f.html',
 'wiki-small//en/articles/r/o/x/ROX_Desktop_b4f8.html',
 'wiki-small//en/articles/k/a/o/KAOS_(software_development)_665f.html',
 'wiki-small//en/articles/c/d/r/CDRWIN_b7e8.html',
 'wiki-small//en/articles/m/i/c/Microtuner.html',
 'wiki-small//en/articles/p/p/3/PP3_444f.html',
 'wiki-small//en/articles/f/o/r/Fortress_(software).html',
 'wiki-small//en/articles/y/a/s/Yass_(software).html',
 'wiki-small//en/articles/s/c/r/Scramdisk.html',
 'wiki-small//en/articles/e/l/e/Electronic_data_capture.html',
 'wiki-small//en/articles/p/i/r/Piriform.html',
 'wiki-small//en/articles/m/i/k/MiKTeX_fd58.html',
 'wiki-small//en/articles/n/+/_/N+.html',
 'wiki-small//en/articles/s/o/f/SoftRAM_b9df.html',
 'wiki-small//en/articles/c/l/x/CLX_(Common_Lisp)_1911.html',
 'wiki-small//en/articles/u/n/i/Universal_Robotics_0bd1.html',
 'wiki-sm

In [55]:
from scipy.stats import spearmanr
spearmancoefficient, datos = spearmanr(tfidf, bm25)
print("Coeficiente de correlación de Spearman:", spearmancoefficient,)

Coeficiente de correlación de Spearman: 0.9999999999999999
